In [ ]:
import numpy as np
import QuantLib as ql
import csv
import random
import pandas as pd

In [ ]:
today = ql.Date(1,1,2022)
ql.Settings.instance().setEvaluationDate(today)
day_counter = ql.Actual365Fixed()

In [ ]:
def fdm(strike, forward, tenor, alpha, beta, nu, rho, t_grid=20, f_grid=200, x_grid=25, scaling_factor=1.0, eps=.0001):
    maturity_date = today + ql.Period(int(round(365 * tenor)), ql.Days)
    r_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, day_counter))
    vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(0, ql.NullCalendar(), 0.0, day_counter))
    bs_process = ql.GeneralizedBlackScholesProcess(ql.QuoteHandle(ql.SimpleQuote(forward)), r_ts, r_ts, vol_ts)
    option = ql.VanillaOption(ql.PlainVanillaPayoff(ql.Option.Call, strike), ql.EuropeanExercise(maturity_date))
    option.setPricingEngine(ql.FdSabrVanillaEngine(forward, alpha, beta, nu, rho, r_ts, t_grid, f_grid, x_grid, 0, scaling_factor, eps))
    return option.impliedVolatility(option.NPV(), bs_process, 1.0e-4, 1000, 1.0e-5, 1.0)

In [ ]:
file_name = 'vol_data/fdm_hagan_vol_train.csv'

# "Strike" and "Nu" here are transformed parameters.

# "Strike" is an approximation for the number of std devs up or down w.r.t. ATM vol, assuming beta = 0.
# "Strike_0" = e^(Strike * Alpha * sqrt(Tenor)) is the actual scaled strike price we use for computing imp. vol.
# The effect of this is that we consider larger strike ranges for longer tenors and larger ATM vols.

# "Nu" is the volvol scaled by the square root of time.
# "Nu_0" = Nu / sqrt(Tenor) is the actual volvol we use for computing imp. vol.
# The effect of this is that we consider smaller volvol ranges for longer tenors.

strikes = [-2, -1.75, -1.5, -1.25, -1, -.75, -.5, -.25, 0, .25, .5, .75, 1, 1.25, 1.5, 1.75, 2]
tenors = [0.5, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 21, 24, 27, 30]
alphas = [.001, .01, .02, .03, .04, .05, .06, .07, .08, .09]
betas = [.001, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
nus = [.1, .5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
rhos = [-.4, -.35, -.3, -.25, -.2, -.15, -.1, -.05, 0, .05, .1, .15, .2, .25, .3, .35, .4]

with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho", "Strike_0", "Nu_0", "FDM_vol", "Hagan_vol"])

keys = set()
for i in range(100):
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        while len(keys) < i * 10000:
            strike = random.choice(strikes)
            tenor = random.choice(tenors)
            alpha = random.choice(alphas)
            beta = random.choice(betas)
            nu = random.choice(nus)
            rho = random.choice(rhos)
            strike_0 = np.exp(strike * alpha * tenor**.5)
            nu_0 = nu / tenor**.5
            if (strike, tenor, alpha, beta, nu, rho) not in keys:
                keys.add((strike, tenor, alpha, beta, nu, rho))
                try:
                    fdm_vol = fdm(strike_0, 1, tenor, alpha, beta, nu_0, rho, 50, 800, 40, eps=.00001)
                    if fdm_vol != 1.0e-5:
                        writer.writerow([
                            strike,
                            tenor,
                            alpha,
                            beta,
                            nu,
                            rho,
                            strike_0,
                            nu_0,
                            fdm_vol,
                            ql.sabrVolatility(strike_0, 1, tenor, alpha, beta, nu_0, rho)
                        ])
                except:
                    pass

In [ ]:
file_name = 'vol_data/fdm_hagan_vol_test.csv'

with open(file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Strike", "Tenor", "Alpha", "Beta", "Nu", "Rho", "Strike_0", "Nu_0", "FDM_vol", "Hagan_vol"])

for i in range(5):
    with open(file_name, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        for j in range(10000):
            strike = random.uniform(-2, 2)
            tenor = random.uniform(.5, 30)
            alpha = random.uniform(.001, .09)
            beta = random.uniform(.001, .5)
            nu = random.uniform(.1, 5)
            rho = random.uniform(-.4, .4)
            strike_0 = np.exp(strike * alpha * tenor**.5)
            nu_0 = nu / tenor**.5
            try:
                fdm_vol = fdm(strike_0, 1, tenor, alpha, beta, nu_0, rho, 50, 800, 40, eps=.00001)
                if fdm_vol != 1.0e-5:
                    writer.writerow([
                        strike,
                        tenor,
                        alpha,
                        beta,
                        nu,
                        rho,
                        strike_0,
                        nu_0,
                        fdm_vol,
                        ql.sabrVolatility(strike_0, 1, tenor, alpha, beta, nu_0, rho)
                    ])
            except:
                pass